In [21]:
# @title 1. Install Dependencies & Setup
import os
import json
import pathlib
import io
import time
import requests
import soundfile as sf
# Updated import for Fish Audio SDK v1.2.0
from fish_audio_sdk import Session, TTSRequest, ReferenceAudio

# Create necessary directories
OUTPUT_DIR = 'output'
OUTPUT_AUDIO_FISH_SUBDIR = 'audio/fish' # Folder: output/audio/fish
full_output_fish_path = os.path.join(OUTPUT_DIR, OUTPUT_AUDIO_FISH_SUBDIR)
os.makedirs(full_output_fish_path, exist_ok=True)

print("✅ Dependencies loaded and folders created.")

✅ Dependencies loaded and folders created.


In [22]:
# @title 2. Configuration

# 🔑 REPLACE THIS WITH YOUR ACTUAL FISH AUDIO API KEY
FISH_API_KEY = "c2e0b01dd7854d55b7928a939626aae3"

INPUT_JSON_FILE = 'manifest.json'
AUDIO_WAV_DIR = 'audio_wav'  # Use converted WAV files from previous step

FOX_TEXT = "The quick brown fox jumps over the lazy dog. But what about the man who watched over them? He barely noticed."

print(f"✅ Configuration set. Using API Key: {FISH_API_KEY[:8]}...****")

✅ Configuration set. Using API Key: c2e0b01d...****


In [23]:
# @title 3. Fish Audio Generation Function

def generate_fish_audio(text: str, reference_audio_path: str, reference_text: str, output_path: str):
    """
    Generate audio using Fish Audio API with reference audio and its transcription.
    """
    if not os.path.exists(reference_audio_path):
        raise FileNotFoundError(f"Reference audio not found: {reference_audio_path}")
    
    try:
        session = Session(FISH_API_KEY)
        
        # Read reference audio
        with open(reference_audio_path, "rb") as f:
            ref_audio_bytes = f.read()
        
        # Create request using ReferenceAudio
        audio_content = io.BytesIO()
        
        # IMPORTANT: Use format='wav' to match filename extension and avoid corruption
        # IMPORTANT: Providing reference_text significantly improves quality
        request = TTSRequest(
            text=text,
            format="wav",
            references=[ReferenceAudio(
                audio=ref_audio_bytes,
                text=reference_text
            )]
        )

        for chunk in session.tts(request):
            audio_content.write(chunk)
            
        # Save to file
        audio_content.seek(0)
        with open(output_path, "wb") as f:
            f.write(audio_content.read())
            
        return True
        
    except Exception as e:
        print(f"   ❌ Fish Audio Error: {e}")
        return False

print("✅ Fish Audio generation function defined (WAV format + reference text support).")

✅ Fish Audio generation function defined (WAV format + reference text support).


In [24]:
# @title 4. Run Processing

def process_samples():
    if not os.path.exists(INPUT_JSON_FILE):
        print(f"❌ Error: '{INPUT_JSON_FILE}' not found.")
        return

    with open(INPUT_JSON_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"\n🚀 Processing {len(data['samples'])} samples using Fish Audio (Quality Fixes Applied)...\n")

    for i, sample in enumerate(data['samples']):
        user_id = sample['user_id']
        original_text = sample['text']
        
        # Map to WAV file in audio_wav
        audio_file_orig = sample['audio_file']
        wav_filename = os.path.basename(audio_file_orig).replace('.webm', '.wav').replace('.m4a', '.wav').replace('.ogg', '.wav')
        ref_audio_path = os.path.join(AUDIO_WAV_DIR, wav_filename)
        
        if not os.path.exists(ref_audio_path):
            print(f"⚠️ Skipping {user_id}: Reference audio not found ({ref_audio_path})")
            continue

        print(f"[{i+1}/{len(data['samples'])}] 🐟 Processing {user_id}...")

        base_name = f"spk-{user_id}-fish"
        orig_filename = f"{base_name}-original.wav"
        new_filename = f"{base_name}-new.wav"
        
        out_path_orig = os.path.join(full_output_fish_path, orig_filename)
        out_path_new = os.path.join(full_output_fish_path, new_filename)

        # Force overwrite by checking and deleting if needed (or just generated if not exist)
        # Given the previous issues, we should regenerate these even if they exist

        # Generate Original Text (Re-cloning the original for quality check)
        if generate_fish_audio(original_text, ref_audio_path, original_text, out_path_orig):
            print(f"   ✅ {orig_filename}")
        
        time.sleep(0.3)

        # Generate Fox Text
        if generate_fish_audio(FOX_TEXT, ref_audio_path, original_text, out_path_new):
            print(f"   ✅ {new_filename}")
        
        time.sleep(0.3)

    print("\n✅ All Fish Audio processing complete!")

try:
    import fish_audio_sdk
    process_samples()
except ImportError:
    print("❌ 'fish-audio-sdk' not found. Please install it: pip install fish-audio-sdk")


🚀 Processing 30 samples using Fish Audio (Quality Fixes Applied)...

[1/30] 🐟 Processing 153293...
   ✅ spk-153293-fish-original.wav
   ✅ spk-153293-fish-new.wav
[2/30] 🐟 Processing 185885...
   ✅ spk-185885-fish-original.wav
   ✅ spk-185885-fish-new.wav
[3/30] 🐟 Processing 173685...
   ✅ spk-173685-fish-original.wav
   ✅ spk-173685-fish-new.wav
[4/30] 🐟 Processing 112547...
   ✅ spk-112547-fish-original.wav
   ✅ spk-112547-fish-new.wav
[5/30] 🐟 Processing 158899...
   ✅ spk-158899-fish-original.wav
   ✅ spk-158899-fish-new.wav
[6/30] 🐟 Processing 34505...
   ✅ spk-34505-fish-original.wav
   ✅ spk-34505-fish-new.wav
[7/30] 🐟 Processing 188127...
   ✅ spk-188127-fish-original.wav
   ✅ spk-188127-fish-new.wav
[8/30] 🐟 Processing 182175...
   ✅ spk-182175-fish-original.wav
   ✅ spk-182175-fish-new.wav
[9/30] 🐟 Processing 18511...
   ✅ spk-18511-fish-original.wav
   ✅ spk-18511-fish-new.wav
[10/30] 🐟 Processing 66259...
   ✅ spk-66259-fish-original.wav
   ✅ spk-66259-fish-new.wav
[11/30] 

In [26]:
# @title 5. Update Metadata (speakers.ts)

def update_speakers_metadata():
    ts_path = os.path.join(OUTPUT_DIR, 'speakers.ts')
    
    if not os.path.exists(ts_path):
        print(f"❌ {ts_path} not found.")
        return

    # Read existing file
    with open(ts_path, 'r', encoding='utf-8') as f:
        content = f.read()
        
    # Extract JSON part
    equals_idx = content.find('=')
    start_idx = content.find('[', equals_idx)
    end_idx = content.rfind(']') + 1
    
    if equals_idx == -1 or start_idx == -1 or end_idx == 0:
        print("❌ Could not parse JSON from speakers.ts")
        return

    json_str = content[start_idx:end_idx]
    try:
        speakers_data = json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"❌ JSON parse error: {e}")
        return

    print(f"📝 Updating metadata for {len(speakers_data)} speakers...")
    
    update_count = 0
    
    for speaker in speakers_data:
        user_id = speaker['id'].replace('spk-', '')
        
        # Audio files
        orig_filename = f"spk-{user_id}-fish-original.wav"
        new_filename = f"spk-{user_id}-fish-new.wav"
        
        path_orig = os.path.join(full_output_fish_path, orig_filename)
        path_new = os.path.join(full_output_fish_path, new_filename)
        
        if os.path.exists(path_orig) and os.path.exists(path_new):
            # Find fish model
            models = speaker.get('models', [])
            fish_model = next((m for m in models if m['modelId'] == 'fish'), None)
            
            new_fish_data = {
                "modelId": "fish",
                "modelName": "Fish Audio",
                "clonedOriginalAudio": f"/{OUTPUT_AUDIO_FISH_SUBDIR}/{orig_filename}",
                "clonedOriginalText": speaker['originalText'],
                "clonedNewAudio": f"/{OUTPUT_AUDIO_FISH_SUBDIR}/{new_filename}",
                "clonedNewText": FOX_TEXT
            }
            
            if not fish_model:
                if 'models' not in speaker: speaker['models'] = []
                speaker['models'].append(new_fish_data)
                update_count += 1
            else:
                fish_model.update(new_fish_data)
                update_count += 1

    # Write back
    new_content = f"export const speakers: Speaker[] = {json.dumps(speakers_data, indent=2)};\n"
    
    with open(ts_path, 'w', encoding='utf-8') as f:
        f.write(new_content)
        
    print(f"✅ Updated {update_count} speakers with Fish Audio data in {ts_path}")

update_speakers_metadata()

📝 Updating metadata for 150 speakers...
✅ Updated 33 speakers with Fish Audio data in output/speakers.ts
